In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# replace only year to get info for desirable year
response = requests.get("https://www.pro-football-reference.com/years/2018/games.htm")

In [3]:
soup = BeautifulSoup(response.content, features= "html.parser")

In [4]:
boxScores = []
for i in range(len(soup.find_all("tr"))):
    try:
        boxScores.append(soup.find_all("tr")[i].find_all("a")[-1].get('href'))
    except IndexError:
        pass

In [5]:
len(boxScores)

267

In [6]:
def refinedInfo(lst,split_el):
    name = ['day','Start Time', 'Stadium', 'Attendance', 'Time of Game']
    dct = dict(zip(name,[0]*len(name)))
    for n in lst:
        t = n.split(split_el) 
        if 'Start Time' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Attendance' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Stadium' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Game' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'day' in n:
            dct['day'] = n
    
    
            
    return list(dct.values())

In [7]:
col_name = ['Team_1','Team_1_score','Team_2','Team_2_score','Day&Date','Start Time','Stadium','Attendance', 'Time of Game']
df = pd.DataFrame(columns= col_name)
for i,half_url in enumerate(boxScores):
    url = "https://www.pro-football-reference.com" + half_url
    response_1 = requests.get(url)
    soup_1 = BeautifulSoup(response_1.content, features= "html.parser")
    var_imd = soup_1.title.get_text().split(',')[0]
    score = [s.get_text().replace('\n','') for s in soup_1.find_all('div',attrs= {'class':"scores"})]
    if ' at' in var_imd:
        teams = var_imd.split(' at')[:2]
        txt_1 = soup_1.find_all("div", attrs= {'class':"scorebox_meta"})
        imd = [j.get_text() for j in txt_1[0].find_all("div")]
        imd_2 = refinedInfo(imd[:-1],': ')
        df.loc[i] = [teams[0]] + [score[1]] + [teams[1]] + [score[0]] +imd_2
    elif ' vs. ' in var_imd:
        teams = var_imd.split(' vs. ')[:2]
        txt_1 = soup_1.find_all("div", attrs= {'class':"scorebox_meta"})
        imd = [j.get_text() for j in txt_1[0].find_all("div")]
        imd_2 = refinedInfo(imd[:-1],': ')
        df.loc[i] = [teams[0]] + [score[1]] + [teams[1]] + [score[0]] +imd_2

In [8]:
df

,Team_1,Team_1_score,Team_2,Team_2_score,Day&Date,Start Time,Stadium,Attendance,Time of Game
0,Atlanta Falcons,12,Philadelphia Eagles - September 6th,18,"Thursday Sep 6, 2018",8:20pm,Lincoln Financial Field,"69,696",3:19
1,Buffalo Bills,3,Baltimore Ravens - September 9th,47,"Sunday Sep 9, 2018",1:00pm,M&T Bank Stadium,"70,591",3:21
2,Tennessee Titans,20,Miami Dolphins - September 9th,27,"Sunday Sep 9, 2018",1:00pm,Hard Rock Stadium,"65,184",7:08
3,Pittsburgh Steelers,21,Cleveland Browns - September 9th,21,"Sunday Sep 9, 2018",1:00pm,FirstEnergy Stadium,"67,431",3:54
4,Houston Texans,20,New England Patriots - September 9th,27,"Sunday Sep 9, 2018",1:00pm,Gillette Stadium,"65,878",3:12
5,San Francisco 49ers,16,Minnesota Vikings - September 9th,24,"Sunday Sep 9, 2018",1:00pm,U.S. Bank Stadium,"66,673",3:12
6,Jacksonville Jaguars,20,New York Giants - September 9th,15,"Sunday Sep 9, 2018",1:00pm,MetLife Stadium,"77,992",3:04
7,Tampa Bay Buccaneers,48,New Orleans Saints - September 9th,40,"Sunday Sep 9, 2018",1:00pm,Mercedes-Benz Superdome,"73,038",3:13
8,Cincinnati Bengals,34,Indianapolis Colts - September 9th,23,"Sunday Sep 9, 2018",1:00pm,Lucas Oil Stadium,"58,699",3:04
9,Kansas City Chiefs,38,Los Angeles Chargers - September 9th,28,"Sunday Sep 9, 2018",4:05pm,StubHub Center,"25,351",3:08


In [9]:
df['Team_1'] = df['Team_1'].apply(lambda x: x.split('-')[-1])
df['Team_2'] = df['Team_2'].apply(lambda x: x.split('-')[0])

In [10]:
df

,Team_1,Team_1_score,Team_2,Team_2_score,Day&Date,Start Time,Stadium,Attendance,Time of Game
0,Atlanta Falcons,12,Philadelphia Eagles,18,"Thursday Sep 6, 2018",8:20pm,Lincoln Financial Field,"69,696",3:19
1,Buffalo Bills,3,Baltimore Ravens,47,"Sunday Sep 9, 2018",1:00pm,M&T Bank Stadium,"70,591",3:21
2,Tennessee Titans,20,Miami Dolphins,27,"Sunday Sep 9, 2018",1:00pm,Hard Rock Stadium,"65,184",7:08
3,Pittsburgh Steelers,21,Cleveland Browns,21,"Sunday Sep 9, 2018",1:00pm,FirstEnergy Stadium,"67,431",3:54
4,Houston Texans,20,New England Patriots,27,"Sunday Sep 9, 2018",1:00pm,Gillette Stadium,"65,878",3:12
5,San Francisco 49ers,16,Minnesota Vikings,24,"Sunday Sep 9, 2018",1:00pm,U.S. Bank Stadium,"66,673",3:12
6,Jacksonville Jaguars,20,New York Giants,15,"Sunday Sep 9, 2018",1:00pm,MetLife Stadium,"77,992",3:04
7,Tampa Bay Buccaneers,48,New Orleans Saints,40,"Sunday Sep 9, 2018",1:00pm,Mercedes-Benz Superdome,"73,038",3:13
8,Cincinnati Bengals,34,Indianapolis Colts,23,"Sunday Sep 9, 2018",1:00pm,Lucas Oil Stadium,"58,699",3:04
9,Kansas City Chiefs,38,Los Angeles Chargers,28,"Sunday Sep 9, 2018",4:05pm,StubHub Center,"25,351",3:08
